#Импортируем библиотеки

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

#Загружаем данные

In [ ]:
df = pd.read_csv('Data.csv').drop('Id', axis=1)

In [ ]:
df.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [ ]:
# класс для построения правил ишибучи и расчетов достоверности

class Ishibuchi:
# класс выполняет обучение и предсказание на основе набора правил и данных.

    """
    Инициализация объекта класса
    Входные параметры: данные, имя переменной моделирования, часть тестового набора,
    часть набора проверки, список терминов в порядке возрастания (например, BS, MS, S, M, B, MB, BB)
    """

    def __init__(self, rules, class_len):
        self.rule_base = rules
        self.class_len = class_len

# метод выполняет обучение модели по переданным тренировочным данным, а также описаниям терминов
    def fit(self, X_train, y_train, *terms):
        self.terms = list(terms)

        self.variable_terms = self.terms_definition(X_train)

        self.rule_class_cf_assign(X_train, y_train, self.rule_base)

        return self
# метод определяет диапазоны значений для каждого термина на основе квантилей данных тренировочного набора.

    """
    Определение каждого переменного термина
    Входные параметры:
    Возврат: набор значений терминов для каждой переменной в наборе данных.
    """
# метод определяет диапазоны значений для каждого термина на основе квантилей данных тренировочного набора.
    def terms_definition(self, train_set):
        terms = self.terms
        terms_dict = {}
        quantiles = np.arange(0., 1., 1 / len(terms))[1:]

        for var in train_set.columns:
            terms_dict[var] = {k:None for k in terms}
            terms_quants = np.quantile(np.array(train_set[var]), quantiles)
            for term, quant in zip(terms, terms_quants):
                terms_dict[var][term] = quant

        return terms_dict


    """
    Функция, которая возвращает соответствующую функцию на основе текущего переменного термина в функции.
    Входные данные: термин переменной в правиле.
    Возврат: соответствующая функция
    """
# определяет функции принадлежности для каждого термина.
    def membership_function(self, term):

        def too_small(x, quant, right_border):
            if x >= right_border:
                return 0
            elif x <= quant:
                return 1
            else:
                return (right_border - x) / (right_border - quant)

        def too_big(x, quant, left_border):
            if x <= left_border:
                return 0
            if x >= quant:
                return 1
            else:
                return (x - left_border) / (quant - left_border)

        def dc_term():
            return 1

        def other_terms(x, quant, left_border, right_border):
            if (x <= left_border) | (x >= right_border):
                return 0
            elif left_border < x <= quant:
                return (x - left_border) / (quant - left_border)
            elif quant < x < right_border:
                return (right_border - x) / (right_border - quant)

        if term == self.terms[0]:
            return too_small
        elif term == self.terms[-2]:
            return too_big
        elif term == self.terms[-1]:
            return dc_term
        else:
            return other_terms


    """
    Функция для подсчета членства по всему правилу
    Входные параметры: правило, набор переменных
    Вывод: минимальная функция принадлежности по правилу.
    """
# метод вычисляет значение принадлежности для каждого правила исходя из переданных значений
    def count_membership_over_the_rule(self, rule:list, X):
        membership_values = []
        for t, v in zip(rule, X.keys()):
            member_func = self.membership_function(t)
            quant_index = self.terms.index(t)
            values = list(self.variable_terms[v].values())
            quant = values[quant_index]
            if 0 < quant_index < len(self.terms) - 2:
                left_border = values[quant_index - 1]
                right_border = values[quant_index + 1]

                membership_values.append(member_func(X[v], quant, left_border, right_border))
            elif quant_index == 0:
                right_border = values[quant_index + 1]

                membership_values.append(member_func(X[v], quant, right_border))

            elif quant_index == len(self.terms) - 2:
                left_border = values[quant_index - 1]

                membership_values.append(member_func(X[v], quant, left_border))

            else:
                membership_values.append(member_func())
        members_wo_zeros = [x for x in membership_values if x != 0]
        return min(members_wo_zeros) if len(members_wo_zeros) > 0 else 0


    """
    Функция для подсчета среднего членства в определенных class_samples
    Входные параметры: метка класса, правило из базы правил.
    Возврат: означает членство в классе
    """
# метод вычисляет суммарное значение принадлежности для каждого класса на основе правила и тренировочного набора.
    def class_membership(self, X_train, y_train, class_, rule):
        class_set = X_train.iloc[y_train[y_train == class_], :]
        membership_sum = 0

        for i in range(class_set.shape[0]):
            membership_sum += self.count_membership_over_the_rule(rule, class_set.iloc[i,:])

        return membership_sum / class_set.shape[0]


    """
    Cчетчик CF
    Входные параметры: словарь каждого бета-класса, истинная метка класса.
    """
# метод вычисляет значимость класса с помощью коэффициента достоверности для каждого класса.
    def cf_counter(self, class_betas_dict, true_class):
        true_class_beta = class_betas_dict[true_class]
        others_betas_mean = sum([class_betas_dict[x] for x in class_betas_dict.keys() if x != true_class]) / (self.class_len - 1)
        return (true_class_beta - others_betas_mean) / sum([class_betas_dict[x] for x in class_betas_dict.keys()])


    """
    Функция для назначения определенного класса каждому правилу в базе правил.
    Входные параметры: база правил
    """
# метод присваивает каждому правилу класс и коэффициент достоверности на основе тренировочного набора.
    def rule_class_cf_assign(self, X_train, y_train, rule_base):
        class_rule_assign = {}

        for rule in rule_base:
            betas = {class_:None for class_ in y_train.unique()}
            rule_s = ' '.join(rule)
            class_rule_assign[rule_s] = {}
            for class_ in betas.keys():
                betas[class_] = self.class_membership(X_train, y_train, class_, rule)

            class_rule_assign[rule_s]['class'] = list(betas.keys())[np.argmax(list(betas.values()))]
            class_rule_assign[rule_s]['cf'] = self.cf_counter(betas, class_rule_assign[rule_s]['class'])

        self.class_rule_assignment = class_rule_assign


    """
    Функция прогнозирования класса
    """
# метод предсказывает класс для новых данных X на основе правил и коэффициентов достоверности.
    def predict(self, X):
        best_class = None
        best_metric = -1
        for rule in self.class_rule_assignment.keys():
            div_rule = rule.split()
            metric = self.count_membership_over_the_rule(div_rule, X) * self.class_rule_assignment[rule]['cf']
            if metric > best_metric:
                best_metric = metric
                best_class = self.class_rule_assignment[rule]['class']

        return best_class

In [ ]:
   '''
   Теперь класс Pittsburg, который выполняет генетический алгоритм для настройки и оптимизации модели.
   Тут инициализируются параметры алгоритма и создается начальная популяция правил
   '''

class Pittsburg:

    def __init__(self, X, y, *terms, initial_population=250, individual_length=10, mutation_probability=0.01, generation_length=100):
        self.generation_length=generation_length
        self.individual_length = individual_length
        self.initial_population = initial_population
        self.mutation_prob = mutation_probability
        self.X = X
        self.y = y
        self.terms = terms
        self.class_number = len(y.unique())

        self.population = [[np.random.choice(self.terms, self.X.shape[1])
                            for _ in range(individual_length)]
                            for _ in range(initial_population)]

# метод создает объекты ишибучи для каждого правила в популяции.
    def build_ishis(self):
        individs_dict = {}
        for i, ind in enumerate(self.population):
            individs_dict[i] = {'rules':ind,
                                'ishi':Ishibuchi(ind, self.class_number).fit(self.X, self.y, *self.terms)}

        self.individuals_dict = individs_dict

# метод вычисляет число правильно классифицированных элементов для данного правила.
    def NCP(self, individual):
        ncp = 0
        predictions = []
        for i in range(self.X.shape[0]):
            predictions.append(individual.predict(self.X.iloc[i, :]))
        number_of_correct = 0
        for p_class, t_class in zip(predictions, self.y):
            if p_class == t_class:
                number_of_correct += 1
        return number_of_correct
# метод вычисляет NCP для каждого правила в популяции и присваивает их соответствующим индивидуумам.
    def assign_ncp_to_individuals(self):
        for individ_key in self.individuals_dict.keys():
            self.individuals_dict[individ_key]['NCP'] = self.NCP(self.individuals_dict[individ_key]['ishi'])
# метод возвращает список значений NCP из индивидуумов.
    def take_ncps(self):
        ncps = []
        for individ_key in self.individuals_dict.keys():
            ncps.append(self.individuals_dict[individ_key]['NCP'])
        return ncps
# метод вычисляет вероятности выбора каждого индивидуума в качестве родителя.
    def assign_parent_probabilities(self):
        ncps = self.take_ncps()
        min_ncp = min(ncps)
        sum_ncp = sum(ncps)
        division = (sum_ncp - len(ncps)*min_ncp)
        for individ_key in self.individuals_dict.keys():
            self.individuals_dict[individ_key]['Parent_Probability'] = (self.individuals_dict[individ_key]['NCP'] - min_ncp) / (1 if division == 0 else division)
        return division

# метод выбирает индекс второго родителя из оставшихся индивидуумов.
    def choose_second_parent(self, first_parent_ind, parents_indecies):
        parents_indecies = list(parents_indecies.copy())
        parents_indecies.remove(first_parent_ind)

        second_parent = None
        while second_parent is None:
            parent_prob = np.random.uniform()
            for parent_ind in parents_indecies:
                if self.individuals_dict[parent_ind]['Parent_Probability'] >= parent_prob:
                    second_parent = parent_ind
                    break
        return second_parent

# метод выполняет операцию скрещивания двух родителей, создавая новое правило.
    def crossover(self, parent_one, parent_two):
        child_one = parent_one.copy()
        child_two = parent_two.copy()
        child = [child_one[j] if np.random.uniform() < 0.5 else child_two[j] for j in range(len(child_one))]
        return child

# метод применяет операцию скрещивания к популяции индивидуумов.
    def apply_uniform_crossover(self):
        new_population = []
        parents_indecies = list(self.individuals_dict.keys())
        for parent_ind in parents_indecies:
            parent_one = self.individuals_dict[parent_ind]['rules']
            parent_two = self.individuals_dict[self.choose_second_parent(parent_ind, parents_indecies)]['rules']

            child = self.crossover(parent_one, parent_two)

            new_population.append(child)

        return new_population

# метод мутации к новой популяции.
    def apply_mutation(self, new_population):
        for i in range(len(new_population)):
            for j in range(len(new_population[i])):
                mut = np.random.uniform()
                if mut <= self.mutation_prob:
                    new_population[i][j] = np.random.choice(self.terms, self.X.shape[1])

# метод создает новую популяцию путем применения скрещивания и мутации, а также сохраняет лучший индивидуум из предыдущего поколения.
    def new_generation(self):
        new_population = self.apply_uniform_crossover()
        self.apply_mutation(new_population)
        elite_index = np.argmax([x['NCP'] for x in self.individuals_dict.values()])
        random_from_new_population = np.random.randint(len(new_population))

        new_population[random_from_new_population] = self.individuals_dict[elite_index]['rules']

        del self.population
        del self.individuals_dict

        self.population = new_population

# метод выполняет генерацию новых поколений популяции и возвращает лучший индивидуум.
    def make_generations(self):
        print('Начало процесса')
        for i in range(1, self.generation_length + 1):
            print(f'/////////////////Generation {i - 1} start/////////////////////')
            print('Building population')
            self.build_ishis()
            print('Assigning ncps')
            self.assign_ncp_to_individuals()
            if i == self.generation_length:
                break
            print('Assigning initial probabilities')
            division = self.assign_parent_probabilities()
            print(f'Generation {i}. MAX NCP is : {np.max([x["NCP"] for x in self.individuals_dict.values()])}')
            if division == 0:
                print('All the individuals are same')
                break
            self.new_generation()
        return self.individuals_dict[np.argmax([x['NCP'] for x in self.individuals_dict.values()])]['ishi']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('quality', axis=1), df['quality'],
                                                    test_size=0.2, stratify=df['quality'])

In [ ]:
ishibuchi = Pittsburg(X_train, y_train, 'TS', 'S', 'M', 'B', 'TB', 'DC',
                      initial_population=20,
                      individual_length=50,
                      generation_length=20) #Возьму 20 для быстроты работы

In [ ]:
best_individ = ishibuchi.make_generations()

Start genetic process
/////////////////Generation 0 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities
Generation 1. MAX NCP is : 208
/////////////////Generation 1 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities
Generation 2. MAX NCP is : 208
/////////////////Generation 2 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities
Generation 3. MAX NCP is : 209
/////////////////Generation 3 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities
Generation 4. MAX NCP is : 209
/////////////////Generation 4 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities
Generation 5. MAX NCP is : 267
/////////////////Generation 5 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities
Generation 6. MAX NCP is : 267
/////////////////Generation 6 start/////////////

Assigning ncps
Assigning initial probabilities
Generation 53. MAX NCP is : 367
/////////////////Generation 53 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities
Generation 54. MAX NCP is : 369
/////////////////Generation 54 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities
Generation 55. MAX NCP is : 369
/////////////////Generation 55 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities
Generation 56. MAX NCP is : 369
/////////////////Generation 56 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities
Generation 57. MAX NCP is : 371
/////////////////Generation 57 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities
Generation 58. MAX NCP is : 371
/////////////////Generation 58 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities
Generation 

Assigning ncps
Assigning initial probabilities
Generation 106. MAX NCP is : 379
/////////////////Generation 106 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities
Generation 107. MAX NCP is : 379
/////////////////Generation 107 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities
Generation 108. MAX NCP is : 379
/////////////////Generation 108 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities
Generation 109. MAX NCP is : 379
/////////////////Generation 109 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities
Generation 110. MAX NCP is : 381
/////////////////Generation 110 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities
Generation 111. MAX NCP is : 381
/////////////////Generation 111 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities

Assigning ncps
Assigning initial probabilities
Generation 158. MAX NCP is : 388
/////////////////Generation 158 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities
Generation 159. MAX NCP is : 388
/////////////////Generation 159 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities
Generation 160. MAX NCP is : 389
/////////////////Generation 160 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities
Generation 161. MAX NCP is : 389
/////////////////Generation 161 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities
Generation 162. MAX NCP is : 389
/////////////////Generation 162 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities
Generation 163. MAX NCP is : 389
/////////////////Generation 163 start/////////////////////
Building population
Assigning ncps
Assigning initial probabilities

In [ ]:
predictions = []
for i in range(X_test.shape[0]):
    predictions.append(best_individ.predict(X_test.iloc[i, :]))

In [ ]:
true_answers = 0
for pr, test in zip(predictions, y_test):
    if pr == test:
        true_answers += 1

In [ ]:
print(f'Точность работы алгоритма: {round(true_answers * 100/y_test.shape[0])} %')

Точность работы алгоритма: 43 %


## Низкая точность это алгоритма может быть связана с несколькими возможными причинами:

* Недостаточное количество обучающих данных: Если объем данных, на которых обучается алгоритм, слишком мал, то алгоритм может не иметь достаточно информации для того, чтобы выучить правильные закономерности и принимать точные решения.

* Переобучение (overfitting): Если модель слишком сложная или имеет слишком много параметров, она может "запоминать" обучающие данные вместо того, чтобы обобщать их основные закономерности. Это может привести к плохим результатам на новых данных.

* Недостаточно эффективные признаки: Если выбранные признаки не содержат достаточно информации, чтобы корректно классифицировать данные, то точность алгоритма может оставаться низкой.

* Неверно настроенные гиперпараметры: Некоторые алгоритмы машинного обучения имеют гиперпараметры, которые необходимо правильно настроить для достижения оптимальной производительности. Если гиперпараметры не правильно настроены, это может привести к плохим результатам.

* Некорректная предобработка данных: Неправильная обработка или недостаточная чистка данных перед обучением алгоритма также может привести к низкой точности.

### Чтобы улучшить точность питсбургского алгоритма, рекомендуется провести анализ и проверить вышеперечисленные возможные причины. Может потребоваться изменить подход к обработке данных, выбору признаков, настройке модели или увеличить объем обучающих данных для более точного обучения алгоритма.